In [180]:
import ast

%matplotlib inline
import numpy as np
import pandas as pd
import resqpy
import xtgeo
from fmu.sumo.explorer import Explorer
from resqpy.crs import Crs
from resqpy.model import Model, new_model
from resqpy.surface import Mesh

sumo = Explorer("dev")

In [181]:
def hash_spec(surface):
    return hash(frozenset(surface.spec.items()))


def create_crs(model, surface, i=""):
    x_offset = surface.spec["xori"]
    y_offset = surface.spec["yori"]
    rotation = surface.spec["rotation"]
    z_inc_down = bool(
        surface.spec["yflip"]
    )  # Determines "handedness" of coordinate system https://en.wikipedia.org/wiki/Right-hand_rule#Coordinates
    title = "Surface Coordinate Reference System " + str(i)

    return Crs(
        model,
        x_offset=x_offset,
        y_offset=y_offset,
        rotation=rotation,
        z_inc_down=z_inc_down,
        title=title,
    )

Find a case and create a model

In [182]:
case = sumo.get_case_by_uuid("06e53a20-3384-4c0e-a9cc-202bebedf7c4")

model = Model(
    epc_file="ensemble.epc",
    new_epc=True,
    create_basics=True,
    create_hdf5_ext=True,
)

Filters telling what the ensemble should contain

In [183]:
filter_by_name = False

iterations = ["iter-0"]
tagnames = ["DS_extract_geogrid", "apstrend_aps_Crevasse_Average"]
names = (not filter_by_name) or ["VOLANTIS GP. Top", "Volon Fm. Top"]

Add all objects (surfaces) which fit the filters into the RESQML model

In [184]:
meshes = []
crss = {}
surfaces = case.surfaces.filter(
    iteration=iterations, tagname=tagnames, name=names
)
for surface in surfaces:
    # Check that object is a realization
    if surface.realization is None:
        continue

    print(surface.name, "-", surface.realization)

    # Generate CRS for object
    if hash_spec(surface) not in crss:
        crss[hash_spec(surface)] = create_crs(
            model, surface, i=len(crss.keys())
        )

    # Retrieve the surface's respective crs
    crs = crss[hash_spec(surface)]

    # Create the mesh for the object
    regsurf = xtgeo.surface_from_file(surface.blob)
    regsurf.to_numpy().fill_value = surface.spec["undef"]

    # Here xy(z) -> ij where i = x, j = y dirs
    origin = (0, 0, 0)
    ni = surface.spec["nrow"]
    nj = surface.spec["ncol"]
    dxyz_dij = np.array(
        [[surface.spec["xinc"], 0, 0], [0, surface.spec["yinc"], 0]]
    )
    z_values = regsurf.to_numpy()
    crs_uuid = crs.uuid
    title = "Surface Mesh"

    mesh = Mesh(
        model,
        z_values=z_values,
        origin=origin,
        ni=ni,
        nj=nj,
        dxyz_dij=dxyz_dij,
        crs_uuid=crs_uuid,
        title=title,
    )
    meshes.append(mesh)

    # Append fmu metadata dict to the mesh
    extra_metadata = surface.metadata
    extra_metadata["uuid"] = surface.uuid
    mesh.append_extra_metadata(extra_metadata)

VOLANTIS GP. Base - 0
VOLANTIS GP. Top - 1
Therys Fm. Top - 2
Volon Fm. Top - 0
Therys Fm. Top - 0
Volon Fm. Top - 2
Therys Fm. Top - 1
VOLANTIS GP. Top - 2
VOLANTIS GP. Base - 2
VOLANTIS GP. Top - 0
VOLANTIS GP. Base - 1
Volon Fm. Top - 1
Valysar Fm. - 1
Valysar Fm. - 2
Valysar Fm. - 0


Write and output the current model

In [185]:
# Write all metadata to epc file
for crs in crss.to_numpy()():
    crs.create_xml()

for mesh in meshes:
    mesh.create_xml()

model.store_epc("ensemble.epc")

# Write data to hdf5 file
for mesh in meshes:
    mesh.write_hdf5()

model.create_hdf5_ext(file_name="ensemble.h5")

<Element {http://www.energistics.org/energyml/data/commonv2}EpcExternalPartReference at 0x22d58353540>